In [1]:
# Mouse L2/3 SST (Som) cell
# http://celltypes.brain-map.org/mouse/experiment/electrophysiology/476686112
specimen_id = 476686112
model_type = 'perisomatic'#or 'all active'
work_dir = 'OriginalFromAllenDB'

In [2]:
from allensdk.model.biophys_sim.config import Config
from allensdk.model.biophysical.utils import Utils
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("ACT")
from act.passive import ACTPassiveModule

In [3]:
from allensdk.api.queries.biophysical_api import BiophysicalApi
from allensdk.model.biophys_sim.config import Config
from allensdk.model.biophysical.utils import Utils

In [4]:

os.chdir(work_dir)

In [5]:
# Compile the modfiles

!nrnivmodl modfiles > /dev/null 2>&1

In [6]:
import sys, os
from functools import wraps

def suppress_output(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        devnull = open(os.devnull, 'w')
        old_stdout, old_stderr = sys.stdout, sys.stderr
        sys.stdout = sys.stderr = devnull
        try:
            return fn(*args, **kwargs)
        finally:
            sys.stdout, sys.stderr = old_stdout, old_stderr
            devnull.close()
    return wrapper

In [7]:

@suppress_output
def build_cell(soma_diam_multiplier = 1,using_hoc=False,hoc_filename=None,cell_name=None):

    if using_hoc:
      from neuron import h
      h.load_file(hoc_filename)
      return getattr(h, cell_name)()
    # Create the h object
    description = Config().load('manifest.json')
    utils = Utils(description)
    h = Utils.h
    # Convert all "value" attributes to floats
    for dict in utils.description.data['genome']:
        for key, value in dict.items():
            if key == 'value': dict[key] = float(value)
    # Configure morphology
    morphology_path = description.manifest.get_path('MORPHOLOGY')
    utils.generate_morphology(morphology_path.encode('ascii', 'ignore'))
    utils.load_cell_parameters()
    # To match PP
    h.soma[0].diam = h.soma[0].diam * soma_diam_multiplier

    return h

In [8]:
def create_syns(syn_point_proc,hobject, seg,dend_sec_no, nsyn):
  syns =  [syn_point_proc(seg, sec=hobject.dend[int(dend_sec_no)]) for i in range(nsyn)]
  return syns

In [13]:

nsyn_exc = int(40 *.04 * 2) #syns in the 40 um windows

def run_sim():

  hobject = build_cell(1)

  #             proximal                                          #distal
  ampa_syns = create_syns(hobject.CPGLUIN,hobject,0.5,32,nsyn_exc) + create_syns(hobject.CPGLUIN,hobject,0.64,33,nsyn_exc)
  nmda_syns =  create_syns(hobject.NMDAIN,hobject,0.5,32,nsyn_exc) + create_syns(hobject.NMDAIN,hobject,0.64,33,nsyn_exc)
  # GABAS = create_syns(h.GABAain,0.5,32,1) + create_syns(h.GABAain,0.64,33,1)


  AMPAS = []
  NMDAS = []
  vecstims_ampa = []
  vecstims_nmda = []

  for i in range(len(ampa_syns)):#assuming same no. ampa and nmda

    vecstims_ampa.append(hobject.VecStim())
    ampaNetCON = hobject.NetCon(vecstims_ampa[-1],ampa_syns[i])
    AMPAS.append(ampaNetCON)


    vecstims_nmda.append(hobject.VecStim())
    nmdaNetCON = hobject.NetCon(vecstims_nmda[-1],nmda_syns[i])
    NMDAS.append(nmdaNetCON)



  # event_times = [[100], [100.5],[101]]
  vecstims_ampa[0].play(hobject.Vector([10]))
  # vecstims_nmda[0].play(hobject.Vector(event_times[0]))
  # vecstims_ampa[1].play(hobject.Vector(event_times[1]))
  # vecstims_nmda[1].play(hobject.Vector(event_times[1]))
  v = hobject.Vector().record(hobject.dend[32](0.5)._ref_v)


  hobject.dt=0.025
  hobject.tstop = 1000
  hobject.finitialize(-75)
  hobject.run()
  #clear NEURON level variables so this cell can be rerun
  V = v.as_numpy()
  return V

In [14]:
v = run_sim()
plt.plot(v)

: 